# Evaluate a real classifier

This code is an example of the use of VADER classifier from NLTK. It is a Naive-Bayes classifier that is trainded with a lexicon and dataset of movie reviews.

Look in the example how the library SKLearn is used to evaulate the classifier.

At the end you have an example on how to use the classifier en custom examples. 


In [24]:
import nltk
from nltk.corpus import movie_reviews
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report, confusion_matrix
import random


In [25]:
# Download required NLTK datasets
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [26]:
# Preprocess the data
stop_words = set(stopwords.words('english'))

def extract_features(words):
    return {word: True for word in words if word.lower() not in stop_words}

In [27]:
# Prepare the dataset
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)  # Shuffle the dataset for better randomness

# Feature extraction
feature_sets = [(extract_features(words), category) for (words, category) in documents]

# Split the data into training and testing sets
train_size = int(len(feature_sets) * 0.8)
train_set, test_set = feature_sets[:train_size], feature_sets[train_size:]

# Train a Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(train_set)

In [28]:
# Evaluate the classifier
print("\nNaive Bayes Classifier Evaluation:")
print(f"Accuracy: {accuracy(classifier, test_set) * 100:.2f}%")
classifier.show_most_informative_features(10)


Naive Bayes Classifier Evaluation:
Accuracy: 70.75%
Most Informative Features
             outstanding = True              pos : neg    =     15.4 : 1.0
                  avoids = True              pos : neg    =     12.5 : 1.0
               ludicrous = True              neg : pos    =     12.1 : 1.0
              brilliance = True              pos : neg    =     11.1 : 1.0
              astounding = True              pos : neg    =     10.4 : 1.0
                captures = True              pos : neg    =     10.3 : 1.0
               addresses = True              pos : neg    =      9.8 : 1.0
            lighthearted = True              pos : neg    =      9.8 : 1.0
                    slip = True              pos : neg    =      9.8 : 1.0
                  stinks = True              neg : pos    =      9.6 : 1.0


In [29]:
# Prepare predictions and true labels for sklearn metrics
y_true = [label for (_, label) in test_set]
y_pred = [classifier.classify(features) for (features, _) in test_set]
# Evaluate using sklearn metrics
print("\nClassification Report:")
print(classification_report(y_true, y_pred))



Classification Report:
              precision    recall  f1-score   support

         neg       1.00      0.40      0.57       196
         pos       0.64      1.00      0.78       204

    accuracy                           0.71       400
   macro avg       0.82      0.70      0.68       400
weighted avg       0.81      0.71      0.68       400



In [30]:
# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# VADER Sentiment Analysis on custom examples
sia = SentimentIntensityAnalyzer()
example_sentences = [
    "I absolutely loved this movie! The acting was fantastic.",
    "This was the worst film I have ever seen.",
    "The plot was predictable, but the cinematography was beautiful.",
    "I wouldn't recommend it. It was boring and too long."
]



Confusion Matrix:
[[ 79 117]
 [  0 204]]


In [31]:
print("\nVADER Sentiment Analysis:")
for sentence in example_sentences:
    score = sia.polarity_scores(sentence)
    sentiment = "positive" if score['compound'] > 0 else "negative"
    print(f"Sentence: {sentence}\nSentiment: {sentiment} (Score: {score['compound']})\n")


VADER Sentiment Analysis:
Sentence: I absolutely loved this movie! The acting was fantastic.
Sentiment: positive (Score: 0.8436)

Sentence: This was the worst film I have ever seen.
Sentiment: negative (Score: -0.6249)

Sentence: The plot was predictable, but the cinematography was beautiful.
Sentiment: positive (Score: 0.7469)

Sentence: I wouldn't recommend it. It was boring and too long.
Sentiment: negative (Score: -0.5283)



# Exercise:

Create your own gold standard and measure Precission, Recall, and F1 manually and with SKLearn to check if the result is the same. 

---

## Creating the gold standard

I´ll create a list of tuples with 14 examples of phrases with positive and negative meanings, using the variable `gold_standard`

In [32]:
# Our gold standard data - manually labeled examples
gold_standard = [
    ("The movie was absolutely fantastic, I loved every minute of it", "pos"),
    ("The actors delivered terrible performances, complete waste of time", "neg"),
    ("While the special effects where great, the story was boring", "neg"),
    ("I found the plot somewhat predictable but still enjoyed it", "pos"),
    ("The plot was interesting, but the performance of the actors made it hard to watch", "neg"),
    ("One of the best movies I have seen this year", "pos"),
    ("Terrible movie, I would love if I could ask for a refund after watching it", "neg"),
    ("The director's vision really shines through in every scene", "pos"),
    ("Despite the large budget, the film feels cheap and rushed", "neg"),
    ("I was moved to tears by the powerful performances", "pos"),
    ("The dialogue was so poorly written it became unintentionally funny", "neg"),
    ("A masterpiece that will be remembered for generations", "pos"),
    ("The cinematography was stunning even though the plot had some holes", "pos"),
    ("Too long and drawn out, I found myself checking my watch repeatedly", "neg"),
    ("An average film that neither impresses nor disappoints", "neg")
]

---
### Separating the gold Standard Data into texts and labels

I do not extract the stopwords as Vader knows them, so I can skip this step

In [33]:
# Extract the first element of the tuples for the texts
texts = [value[0] for value in gold_standard]

# Extract the second element of the tuples for pos neg labels
true_labels = [value[1] for value in gold_standard]

In [34]:
print(texts)

print(true_labels)

['The movie was absolutely fantastic, I loved every minute of it', 'The actors delivered terrible performances, complete waste of time', 'While the special effects where great, the story was boring', 'I found the plot somewhat predictable but still enjoyed it', 'The plot was interesting, but the performance of the actors made it hard to watch', 'One of the best movies I have seen this year', 'Terrible movie, I would love if I could ask for a refund after watching it', "The director's vision really shines through in every scene", 'Despite the large budget, the film feels cheap and rushed', 'I was moved to tears by the powerful performances', 'The dialogue was so poorly written it became unintentionally funny', 'A masterpiece that will be remembered for generations', 'The cinematography was stunning even though the plot had some holes', 'Too long and drawn out, I found myself checking my watch repeatedly', 'An average film that neither impresses nor disappoints']
['pos', 'neg', 'neg', 'p

In [ ]:
print("\nVADER Sentiment Analysis in custom gold Standard:")
for sentence in texts:
    score = sia.polarity_scores(sentence)
    sentiment = "positive" if score['compound'] > 0 else "negative"
    print(f"Sentence: {sentence}\nSentiment: {sentiment} (Score: {score['compound']})\n")